# PassiveSSL Analysis

PassiveSSL tracks SSL/TLS certificates observed on the internet, providing historical data for threat intelligence and security research.

## Key Features
- Certificate history for IP addresses
- Certificate-to-IP mappings
- X.509 certificate details
- Validity period tracking

## Documentation
- API Docs: https://www.circl.lu/services/passive-ssl/
- Client Reference: https://github.com/adulau/crl-monitor/tree/master/client

## Exercises

### Setup Authentication

In [ ]:
import pypssl
import getpass

# Configure PassiveSSL credentials
PASSIVE_SSL_USER = getpass.getpass("PassiveSSL Username: ")
PASSIVE_SSL_KEY = getpass.getpass("PassiveSSL API Key: ")

pssl = pypssl.PyPSSL(basic_auth=(PASSIVE_SSL_USER, PASSIVE_SSL_KEY))

### 1.0 Get Certificate Information for an IP

Query all certificates observed for a specific IP address.

**API Endpoint:** `https://www.circl.lu/v2pssl/query/[ip]`

In [ ]:
ip = '8.8.8.8'
data = pssl.query(ip)

print(f"PassiveSSL Data for {ip}")
print("="*40)
print("Certificates found:")
for cert in data[ip]["certificates"]:
    print(f"  {cert}")

print("\nSubjects:")
for cert_hash, subj_data in data[ip]["subjects"].items():
    print(f"  {cert_hash}")
    for val in subj_data["values"]:
        print(f"    {val}")

### 1.1 Get IPs Associated with a Certificate

Find all IP addresses that have used a specific certificate.

**API Endpoint:** `https://www.circl.lu/pssl/cquery/[certificate_sha1]`

In [ ]:
cert_sha1 = 'c46fed822dadac3f31f9bb4d1a78a1d9eae4567b'
data = pssl.query_cert(cert_sha1)
print(data)

### 1.2 Retrieve X.509 Certificate Details

Fetch the complete certificate data including PEM format.

**API Endpoint:** `https://www.circl.lu/pssl/cfetch/[certificate_sha1]`

In [ ]:
cert_sha1 = 'c46fed822dadac3f31f9bb4d1a78a1d9eae4567b'
data = pssl.fetch_cert(cert_sha1)
print(data["pem"])

### 1.3 Identify Expired Certificates

Check which certificates have passed their expiration date.

In [ ]:
from datetime import datetime, UTC

def check_expired_certs(cert_info):
    now = datetime.now(UTC)
    info = cert_info.get('info', {})
    
    not_after = info.get('not_after')
    fingerprint = info.get('fingerprint', 'N/A')
    
    if not_after and not_after < now:
        return (fingerprint, not_after)
    return None

ip = '8.8.8.8'
data = pssl.query(ip)

print("Checking for expired certificates...")
expired_count = 0

for cert_sha1 in data[ip]['certificates']:
    try:
        cert_info = pssl.fetch_cert(cert_sha1)
        expired = check_expired_certs(cert_info)
        if expired:
            fp, exp_date = expired
            print(f"  Expired: {fp} (expired {exp_date})")
            expired_count += 1
    except Exception as ex:
        print(f"  Error processing {cert_sha1}: {ex}")

if expired_count == 0:
    print("  No expired certificates found.")

### 1.4 List Certificate Issuers

Identify all unique certificate authorities that issued certificates for the IP.

In [ ]:
from collections import Counter

ip = '8.8.8.8'
data = pssl.query(ip)

issuers = []
print("Certificate issuers:")

for cert_sha1 in data[ip]['certificates']:
    try:
        cert_info = pssl.fetch_cert(cert_sha1)
        info = cert_info.get('info', {})
        issuer = info.get('issuer', 'Unknown')
        issuers.append(issuer)
        print(f"  {issuer}")
    except Exception as ex:
        print(f"  Error processing {cert_sha1}: {ex}")

# Count unique issuers
issuer_counts = Counter(issuers)
print(f"\nUnique issuers: {len(issuer_counts)}")
for issuer, count in issuer_counts.items():
    print(f"  {issuer}: {count} certificates")

### 1.5 Certificate Validity Timeline

Visualize the validity periods of all certificates for the IP address.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

ip = '8.8.8.8'
data = pssl.query(ip)

timeline_data = []

for cert_sha1 in data[ip]["certificates"]:
    try:
        cert_info = pssl.fetch_cert(cert_sha1)
        info = cert_info.get('info', {})
        
        not_before = info.get('not_before')
        not_after = info.get('not_after')
        fingerprint = info.get('fingerprint', cert_sha1[:12])
        
        if not_before and not_after:
            timeline_data.append({
                'fingerprint': fingerprint,
                'not_before': not_before,
                'not_after': not_after
            })
    except Exception as ex:
        print(f"Error processing {cert_sha1}: {ex}")

if timeline_data:
    # Create timeline visualization
    fig, ax = plt.subplots(figsize=(12, max(6, len(timeline_data)*0.8)))
    
    dates_before = [mdates.date2num(x['not_before']) for x in timeline_data]
    dates_after = [mdates.date2num(x['not_after']) for x in timeline_data]
    durations = [after - before for before, after in zip(dates_before, dates_after)]
    
    y_pos = range(len(timeline_data))
    
    ax.barh(y_pos, durations, left=dates_before, height=0.6, 
            color='steelblue', alpha=0.7)
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels([d['fingerprint'] for d in timeline_data])
    
    ax.xaxis_date()
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    
    ax.set_xlabel('Certificate Validity Period')
    ax.set_title(f'Certificate Timeline for {ip}')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No valid certificate data found for timeline.")

### Homework - Analize certificates for a given IP
- Exercise: Analyze certificates for an IP
    1. Choose an IP address (e.g., `8.8.8.8`) and query PassiveSSL for its certificates.
    2. For each certificate SHA1 returned, fetch full certificate details and extract: fingerprint, issuer, not_before, not_after.
    3. Use the existing helper (see the expired-check logic) to list certificates that are already expired.
    4. Produce a short summary: total certificates, number expired, unique issuers (with counts).
    5. Visualize the validity periods as a horizontal timeline similar to the timeline cell.

- Deliverables
    - Printed summary (counts, list of expired fingerprints).
    - A bar chart timeline of certificate validity periods.

- Hints
    - Use pssl.query(...) to get the certificate list (see earlier query cells).
    - Use pssl.fetch_cert(...) to get `info` fields (issuer, not_before, not_after).
    - Reuse the expired-check function and timeline plotting code already in the notebook for implementation and formatting.

In [ ]:
# Your code here...